<a href="https://colab.research.google.com/github/gilsonauerswald/IC_Projeto_Stx2a/blob/main/ABRicate_VFDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

# Configuração Estética Global (Estilo Nature/Scientific Reports)
sns.set_style("white")
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['figure.dpi'] = 600

# 1. Carregamento e Processamento de Dados
# O arquivo é carregado assumindo tabulação. Ajustes de índice são feitos para extrair nomes.
df = pd.read_csv('Galaxy33-[ABRicate Summary on dataset 9-16 VFDB].tabular', sep='\t')
df = df.reset_index()

# Identificação da coluna de cepas e criação de grupos
strain_col = [c for c in df.columns if df[c].astype(str).str.contains('E. coli').any()][0]
df.rename(columns={strain_col: 'Strain'}, inplace=True)
df['Group'] = df['Strain'].apply(lambda x: 'O157' if 'O157' in str(x) else 'Não-O157')

# Limpeza e Binarização (0 = Ausente, 1 = Presente)
metadata_cols = ['Strain', '#FILE', 'NUM_FOUND', 'Group', 'level_0', 'level_1']
gene_cols = [c for c in df.columns if c not in metadata_cols and 'Unnamed' not in c]

def clean_gene(val):
    s_val = str(val).strip()
    return 0 if s_val in ['.', '', 'nan'] else 1

binary_df = df.copy()
for col in gene_cols:
    binary_df[col] = binary_df[col].apply(clean_gene)

stx2a_col = next((c for c in gene_cols if 'stx2A' in c), None)

# 2. Geração dos Gráficos

# Gráfico 1: Heatmap de Virulência
heatmap_df = binary_df.sort_values('Group', ascending=False)
present_genes = [c for c in gene_cols if heatmap_df[c].sum() > 0] # Filtra genes totalmente ausentes
lut = {'O157': '#440154', 'Não-O157': '#FDE725'} # Paleta Viridis (Colorblind-friendly)
row_colors = heatmap_df['Group'].map(lut)

g = sns.clustermap(heatmap_df[present_genes], row_cluster=False, col_cluster=True,
                   row_colors=row_colors, cmap="viridis", cbar_pos=(0.02, 0.8, 0.03, 0.15),
                   yticklabels=heatmap_df['Strain'], xticklabels=False, figsize=(14, 8))
g.ax_heatmap.set_ylabel("Isolados", fontsize=12, fontweight='bold')
g.ax_heatmap.set_xlabel(f"Fatores de Virulência ({len(present_genes)} genes)", fontsize=12, fontweight='bold')
g.ax_heatmap.legend(handles=[Patch(facecolor=lut[k], label=k) for k in lut],
                    title='Grupo', loc='upper left', bbox_to_anchor=(1.02, 1))
plt.savefig('heatmap_virulence.png', bbox_inches='tight')
plt.show() # Adicionado para exibir o gráfico

# Gráfico 2: Prevalência de stx2a
if stx2a_col:
    prev_df = (binary_df.groupby('Group')[stx2a_col].mean() * 100).reset_index()
    plt.figure(figsize=(5, 6))
    ax = sns.barplot(x='Group', y=stx2a_col, data=prev_df, palette=lut)
    ax.set_ylabel("Prevalência de stx2a (%)", fontsize=12, fontweight='bold')
    ax.set_xlabel("Grupo", fontsize=12, fontweight='bold')
    ax.set_ylim(0, 110)
    sns.despine()
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')
    plt.savefig('prevalence_stx2a.png', bbox_inches='tight')
    plt.show() # Adicionado para exibir o gráfico

# Gráfico 3: Co-ocorrência
if stx2a_col:
    corr = binary_df[gene_cols].corrwith(binary_df[stx2a_col]).drop(stx2a_col)
    top_corr = corr.sort_values(ascending=False).head(10)
    plt.figure(figsize=(8, 5))
    sns.barplot(x=top_corr.values, y=top_corr.index, palette='magma')
    plt.xlabel("Coeficiente de Correlação (Pearson)", fontsize=12, fontweight='bold')
    plt.title(f"Genes co-ocorrentes com {stx2a_col}", fontsize=14, fontweight='bold')
    sns.despine()
    plt.savefig('cooccurrence_stx2a.png', bbox_inches='tight')
    plt.show() # Adicionado para exibir o gráfico

FileNotFoundError: [Errno 2] No such file or directory: 'Galaxy33-[ABRicate Summary on dataset 9-16 VFDB].tabular'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

# Configuração Estética Global (Estilo Nature/Scientific Reports)
sns.set_style("white")
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['figure.dpi'] = 600

# 1. Carregamento e Processamento de Dados
# O arquivo é carregado assumindo tabulação. Ajustes de índice são feitos para extrair nomes.
df = pd.read_csv('Galaxy33-[ABRicate Summary on dataset 9-16 VFDB].tabular', sep='\t')
df = df.reset_index()

# Identificação da coluna de cepas e criação de grupos
strain_col = [c for c in df.columns if df[c].astype(str).str.contains('E. coli').any()][0]
df.rename(columns={strain_col: 'Strain'}, inplace=True)
df['Group'] = df['Strain'].apply(lambda x: 'O157' if 'O157' in str(x) else 'Não-O157')

# Limpeza e Binarização (0 = Ausente, 1 = Presente)
metadata_cols = ['Strain', '#FILE', 'NUM_FOUND', 'Group', 'level_0', 'level_1']
gene_cols = [c for c in df.columns if c not in metadata_cols and 'Unnamed' not in c]

def clean_gene(val):
    s_val = str(val).strip()
    return 0 if s_val in ['.', '', 'nan'] else 1

binary_df = df.copy()
for col in gene_cols:
    binary_df[col] = binary_df[col].apply(clean_gene)

stx2a_col = next((c for c in gene_cols if 'stx2A' in c), None)

# 2. Geração dos Gráficos

# Gráfico 1: Heatmap de Virulência
heatmap_df = binary_df.sort_values('Group', ascending=False)
present_genes = [c for c in gene_cols if heatmap_df[c].sum() > 0] # Filtra genes totalmente ausentes
lut = {'O157': '#440154', 'Não-O157': '#FDE725'} # Paleta Viridis (Colorblind-friendly)
row_colors = heatmap_df['Group'].map(lut)

g = sns.clustermap(heatmap_df[present_genes], row_cluster=False, col_cluster=True,
                   row_colors=row_colors, cmap="viridis", cbar_pos=(0.02, 0.8, 0.03, 0.15),
                   yticklabels=heatmap_df['Strain'], xticklabels=False, figsize=(14, 8))
g.ax_heatmap.set_ylabel("Isolados", fontsize=12, fontweight='bold')
g.ax_heatmap.set_xlabel(f"Fatores de Virulência ({len(present_genes)} genes)", fontsize=12, fontweight='bold')
g.ax_heatmap.legend(handles=[Patch(facecolor=lut[k], label=k) for k in lut],
                    title='Grupo', loc='upper left', bbox_to_anchor=(1.02, 1))
plt.savefig('heatmap_virulence.png', bbox_inches='tight')
plt.show() # Adicionado para exibir o gráfico

# Gráfico 2: Prevalência de stx2a
if stx2a_col:
    prev_df = (binary_df.groupby('Group')[stx2a_col].mean() * 100).reset_index()
    plt.figure(figsize=(5, 6))
    ax = sns.barplot(x='Group', y=stx2a_col, data=prev_df, palette=lut)
    ax.set_ylabel("Prevalência de stx2a (%)", fontsize=12, fontweight='bold')
    ax.set_xlabel("Grupo", fontsize=12, fontweight='bold')
    ax.set_ylim(0, 110)
    sns.despine()
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')
    plt.savefig('prevalence_stx2a.png', bbox_inches='tight')
    plt.show() # Adicionado para exibir o gráfico

# Gráfico 3: Co-ocorrência
if stx2a_col:
    corr = binary_df[gene_cols].corrwith(binary_df[stx2a_col]).drop(stx2a_col)
    top_corr = corr.sort_values(ascending=False).head(10)
    plt.figure(figsize=(8, 5))
    sns.barplot(x=top_corr.values, y=top_corr.index, palette='magma')
    plt.xlabel("Coeficiente de Correlação (Pearson)", fontsize=12, fontweight='bold')
    plt.title(f"Genes co-ocorrentes com {stx2a_col}", fontsize=14, fontweight='bold')
    sns.despine()
    plt.savefig('cooccurrence_stx2a.png', bbox_inches='tight')
    plt.show() # Adicionado para exibir o gráfico

FileNotFoundError: [Errno 2] No such file or directory: 'Galaxy33-[ABRicate Summary on dataset 9-16 VFDB].tabular'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

# Configuração Estética Global (Estilo Nature/Scientific Reports)
sns.set_style("white")
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['figure.dpi'] = 600

# 1. Carregamento e Processamento de Dados
# O arquivo é carregado assumindo tabulação. Ajustes de índice são feitos para extrair nomes.
df = pd.read_csv('Galaxy33-[ABRicate Summary on dataset 9-16 VFDB].tabular', sep='\t')
df = df.reset_index()

# Identificação da coluna de cepas e criação de grupos
strain_col = [c for c in df.columns if df[c].astype(str).str.contains('E. coli').any()][0]
df.rename(columns={strain_col: 'Strain'}, inplace=True)
df['Group'] = df['Strain'].apply(lambda x: 'O157' if 'O157' in str(x) else 'Não-O157')

# Limpeza e Binarização (0 = Ausente, 1 = Presente)
metadata_cols = ['Strain', '#FILE', 'NUM_FOUND', 'Group', 'level_0', 'level_1']
gene_cols = [c for c in df.columns if c not in metadata_cols and 'Unnamed' not in c]

def clean_gene(val):
    s_val = str(val).strip()
    return 0 if s_val in ['.', '', 'nan'] else 1

binary_df = df.copy()
for col in gene_cols:
    binary_df[col] = binary_df[col].apply(clean_gene)

stx2a_col = next((c for c in gene_cols if 'stx2A' in c), None)

# 2. Geração dos Gráficos

# Gráfico 1: Heatmap de Virulência
heatmap_df = binary_df.sort_values('Group', ascending=False)
present_genes = [c for c in gene_cols if heatmap_df[c].sum() > 0] # Filtra genes totalmente ausentes
lut = {'O157': '#440154', 'Não-O157': '#FDE725'} # Paleta Viridis (Colorblind-friendly)
row_colors = heatmap_df['Group'].map(lut)

g = sns.clustermap(heatmap_df[present_genes], row_cluster=False, col_cluster=True,
                   row_colors=row_colors, cmap="viridis", cbar_pos=(0.02, 0.8, 0.03, 0.15),
                   yticklabels=heatmap_df['Strain'], xticklabels=False, figsize=(14, 8))
g.ax_heatmap.set_ylabel("Isolados", fontsize=12, fontweight='bold')
g.ax_heatmap.set_xlabel(f"Fatores de Virulência ({len(present_genes)} genes)", fontsize=12, fontweight='bold')
g.ax_heatmap.legend(handles=[Patch(facecolor=lut[k], label=k) for k in lut],
                    title='Grupo', loc='upper left', bbox_to_anchor=(1.02, 1))
plt.savefig('heatmap_virulence.png', bbox_inches='tight')
plt.show() # Adicionado para exibir o gráfico

# Gráfico 2: Prevalência de stx2a
if stx2a_col:
    prev_df = (binary_df.groupby('Group')[stx2a_col].mean() * 100).reset_index()
    plt.figure(figsize=(5, 6))
    ax = sns.barplot(x='Group', y=stx2a_col, data=prev_df, palette=lut)
    ax.set_ylabel("Prevalência de stx2a (%)", fontsize=12, fontweight='bold')
    ax.set_xlabel("Grupo", fontsize=12, fontweight='bold')
    ax.set_ylim(0, 110)
    sns.despine()
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')
    plt.savefig('prevalence_stx2a.png', bbox_inches='tight')
    plt.show() # Adicionado para exibir o gráfico

# Gráfico 3: Co-ocorrência
if stx2a_col:
    corr = binary_df[gene_cols].corrwith(binary_df[stx2a_col]).drop(stx2a_col)
    top_corr = corr.sort_values(ascending=False).head(10)
    plt.figure(figsize=(8, 5))
    sns.barplot(x=top_corr.values, y=top_corr.index, palette='magma')
    plt.xlabel("Coeficiente de Correlação (Pearson)", fontsize=12, fontweight='bold')
    plt.title(f"Genes co-ocorrentes com {stx2a_col}", fontsize=14, fontweight='bold')
    sns.despine()
    plt.savefig('cooccurrence_stx2a.png', bbox_inches='tight')
    plt.show() # Adicionado para exibir o gráfico

FileNotFoundError: [Errno 2] No such file or directory: 'Galaxy33-[ABRicate Summary on dataset 9-16 VFDB].tabular'